<a href="https://www.kaggle.com/code/pogscafe/fooocus?scriptVersionId=178698955" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Fooocus notebook for Kaggle

🔗 GitHub link for this notebook - https://github.com/wandaweb/Fooocus-Kaggle

🌎 Connect with us for updates - https://pogscafe.bio.link 

🔗 Fooocus on GitHub - https://github.com/lllyasviel/Fooocus

# Installation

Best settings:
 - Accelerator: GPU P100
 - Environment: Pin to original environment (2023-11-15)
 - Persistence: Files only
 
Make sure the Internet is on 😄

Installation takes about 5 minutes.

In [ ]:
HOME_FOLDER = '/kaggle/working'
TEMP_FOLDER = '/kaggle/temp'
update = False

CKPT_FOLDER = f'{HOME_FOLDER}/Fooocus/models/checkpoints'

!mkdir $TEMP_FOLDER

import os
from os import path

%cd $HOME_FOLDER

if not path.exists('Fooocus'):
    get_ipython().system('git clone https://github.com/lllyasviel/Fooocus.git')
else:
    get_ipython().system(f'find {CKPT_FOLDER} -maxdepth 1 -type l -delete') # delete any symlinks to temp models from a previous run
    
%cd Fooocus

if update:
    get_ipython().system('git pull')
    
!pip install -q -r requirements_versions.txt 
!pip install -q torch torchvision --force-reinstall --index-url https://download.pytorch.org/whl/cu117
!rm -rf /opt/conda/lib/python3.10/site-packages/pytz-2023.3.dist-info
!rm -rf /opt/conda/lib/python3.10/site-packages/PyYAML-6.0.1.dist-info
!pip install -q pyyaml pytz 
!mamba install openssh -y


# Start the WebUI

**Option 1: Starting the Web UI with ngrok**  
* Make sure to put your ngrok token in the Ngrok_token variable. The token can be obtained from https://ngrok.com
* If you have a static domain, put your ngrok domain in the Ngrok_domain variable.
* Wait for the line that says "App started successful. Use the app with http://127.0.0.1:7865/ or 127.0.0.1:7865" 
    * If the port is no longer 7865 in the latest version, please change the port variable as well
* Visit your ngrok URL (either your static domain, or the ngrok url displayed in the output)

In [ ]:
# Starting the Web UI with ngrok

# --- Variables ---

Ngrok_token = "" #@param {type:"string"}
# Put your ngrok token here (obtainable from https://ngrok.com)
# Example: Ngrok_token = "2Fw13n4GcJT12g7mSDUC62cdNGb_5svdjf3Gg5vfhr4nGr5gF"

Ngrok_domain = "" # optional, leave empty if you don't have a domain

port = 7865

# -----------------


!pip install pyngrok

from pyngrok import ngrok, conf
import fileinput
import sys
import gc

gc.collect()

if Ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(port, pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token),
                    bind_tls=True, domain=Ngrok_domain).public_url
  print(srv)
  get_ipython().system(f"python {HOME_FOLDER}/Fooocus/entry_with_update.py ")
else:
  print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the ngrok_token field.')

***   
**Option 2: Starting the Web UI Pinggy**  
* Wait for the line that says "App started successful. Use the app with http://127.0.0.1:7865/ or 127.0.0.1:7865". It might take a minute for this line to show up at the end of the output.   
* Click the link ending with pinggy.link between the smiling emojis

In [ ]:
# Starting the Web UI with pinggy

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = "python $HOME_FOLDER/Fooocus/entry_with_update.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:7865 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            #print(line)
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

***   
**Option 3: Starting the Web UI with Zrok**  


### Install Zrok

In [ ]:
# Install Zrok (only needs to run once)

!mkdir /kaggle/working/zrok
%cd /kaggle/working/zrok
!wget https://github.com/openziti/zrok/releases/download/v0.4.23/zrok_0.4.23_linux_amd64.tar.gz
!tar -xvf ./zrok*.gz 
!chmod a+x /kaggle/working/zrok/zrok 

### Create a Zrok account
Enter your email address in the email variable

In [ ]:
email = '####@gmail.com' # replace with your email

# --------------

cmd = '/kaggle/working/zrok/zrok invite'
log = '/kaggle/working/zrok/log.txt'

!pip install pexpect
!touch $log

import pexpect
import time
child = pexpect.spawn('bash')
child.sendline(f'{cmd} | tee {log}')
child.expect('enter and confirm your email address...')
time.sleep(1); child.sendline(email); time.sleep(1); child.send(chr(9)); time.sleep(1)
child.sendline(email); time.sleep(1); child.send('\n'); time.sleep(1); child.send(chr(9))
time.sleep(1); child.send('\r\n'); time.sleep(2); child.close()
!cat $log
!rm $log

### Enable Zrok 
Paste your Zrok token in the token variable

In [ ]:
# Enable Zrok (Run once per session)
# Paste your Zrok token in the token variable

token = ""
!chmod a+x /kaggle/working/zrok/zrok
!/kaggle/working/zrok/zrok enable $token

### 🖌 Start the WebUI with Zrok

In [ ]:
# Start the WebUI with Zrok
%cd /kaggle/working/StoryDiffusion
command = f'python {HOME_FOLDER}/Fooocus/entry_with_update.py'
port = '7865'
# ------------------------

cmd = f'{command} & /kaggle/working/zrok/zrok share public http://localhost:{port} --headless'
get_ipython().system(cmd)

# Install Models

In [ ]:
# Install a model to permanent storage
# Make sure Persistence is set to "Files only" or "Variables and Files"
model_url = 'https://civitai.com/api/download/models/169740?type=Model&format=SafeTensor&size=full&fp=fp16'
model_name = 'ZavyChroma.safetensors'

%cd $CKPT_FOLDER
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

In [ ]:
# Install a LoRA to permanent storage
model_url = 'https://civitai.com/api/download/models/505826'
model_name = 'PaperBook.safetensors'

LORA_FOLDER = HOME_FOLDER + '/Fooocus/models/loras'
%cd $LORA_FOLDER
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

In [ ]:
# Install a model to temporary storage
model_url = 'https://civitai.com/api/download/models/395107?type=Model&format=SafeTensor&size=pruned&fp=fp16'
model_name = 'Unstable.safetensors'

%cd $TEMP_FOLDER
if not path.exists(f'{CKPT_FOLDER}/{model_name}'):
    get_ipython().system(f'wget -O "{model_name}" "{model_url}"')
    get_ipython().system(f'ln -s {TEMP_FOLDER}/{model_name} {CKPT_FOLDER}/')
else:
    print(f'The file {CKPT_FOLDER}/{model_name} already exists')

## Remove corrupted models

In [ ]:
!ls -la $CKPT_FOLDER
!rm -rf $CKPT_FOLDER/*.corrupted
!ls -la $CKPT_FOLDER

## 
## Delete permanent models and LoRAs

In [ ]:
!rm $HOME_FOLDER/Fooocus/models/checkpoints/*
!rm $HOME_FOLDER/Fooocus/models/loras/*

## File Manager


In [ ]:
# Install File Manager
%cd /kaggle
!wget https://github.com/filebrowser/filebrowser/releases/download/v2.27.0/linux-amd64-filebrowser.tar.gz
!tar xvfz linux-amd64-filebrowser.tar.gz
!chmod a+x /kaggle/filebrowser
!rm /kaggle/config.json
!/kaggle/filebrowser config init > /dev/null
!/kaggle/filebrowser config set --auth.method=noauth > /dev/null
!/kaggle/filebrowser config set --branding.theme=dark > /dev/null
!/kaggle/filebrowser users add admin admin > /dev/null
!/kaggle/filebrowser config export "/kaggle/working/config.json" > /dev/null

In [ ]:
# Run File Manager with Pinggy
%cd /kaggle
        
from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    !/kaggle/filebrowser -c "/kaggle/config.json" & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8080 a.pinggy.io > log.txt > log.txt
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

Sample of creating a shortcut to a temporary lora

In [ ]:
!ln -s "/kaggle/temp/Dark Fantasy.safetensors" "/kaggle/working/Fooocus/models/loras"